In [1]:
# !pip install pyyaml

In [73]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd

In [63]:
job_ids = []

In [64]:
def get_job_ids(total_pages=1):
    for i in tqdm(range(total_pages)):
        # Set the URL you want to webscrape from
        url = 'https://www.linkedin.com/jobs/search?keywords=Software%20Engineer&location=Berlin%2C%20Berlin%2C%20Germany&geoId=&trk=homepage-jobseeker_jobs-search-bar_search-submit&redirect=false&position=1&pageNum={}'.format(i)
        # Connect to the URL
        response = requests.get(url)

        # Parse HTML and save to BeautifulSoup object¶
        soup = BeautifulSoup(response.text, "html.parser")
        
        ## extract job ids from the selected page
        extract_job_ids(soup)

In [65]:
## gets job ids from one page at a time
def extract_job_ids(soup):
    jobs = soup.findAll("li", attrs={"class":"result-card job-result-card result-card--with-hover-state"})
    
    for job in jobs:
        job_ids.append(job["data-id"])

In [66]:
get_job_ids(total_pages=100)

100%|██████████| 100/100 [01:32<00:00,  1.08it/s]


In [67]:
len(job_ids)

2474

In [9]:
for job_id in job_ids:
    url = """
    https://www.linkedin.com/jobs/search?keywords=Software%2BEngineer&location=Berlin%2C%2BBerlin%2C%2BGermany&trk=homepage-jobseeker_jobs-search-bar_search-submit&currentJobId={}&position=2&pageNum=0
    """.format(job_id)
#     print(url)

In [10]:

# for job_id in job_ids:
#     url = """
#     https://www.linkedin.com/jobs/search?keywords=Software%2BEngineer
#     &location=Berlin%2C%2BBerlin%2C%2BGermany
#     &trk=homepage-jobseeker_jobs-search-bar_search-submit
#     &currentJobId={}
#     &position=2
#     &pageNum=0
#     """.format(job_id)
#     # Connect to the URL
#     response = requests.get(url)

#     # Parse HTML and save to BeautifulSoup object¶
#     soup = BeautifulSoup(response.text, "html.parser")
#     print(soup)

## wrote some functions for scraping and also we need to keep the sleep function so that
## server doesn't detect too many requests

In [79]:
def get_job_data(job_id):
    url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}".format(job_id)
    # Connect to the URL
    response = requests.get(url)

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    ## find jd section
    return soup.find("section",attrs={"class":"description"}).text

In [80]:
job_descriptions = []
for job_id in tqdm(job_ids[:5]):
    jd = get_job_data(job_id)
    job_descriptions.append(jd)
    time.sleep(2.4)

100%|██████████| 5/5 [00:14<00:00,  2.93s/it]


In [81]:
pd.DataFrame(job_descriptions, columns=['description'])

,description
0,DescriptionSoftware Development Engineer Gradu...
1,Rust / C++ Software Engineer Do you have aspir...
2,"At a Glance:On-Running, the most exciting and ..."
3,DescriptionCome join an experienced team of en...
4,About The OpportunityWe are seeking a talented...


## for lazy loading 
https://stackoverflow.com/questions/40777864/retrieving-all-information-from-page-beautifulsoup

In [14]:
## for converting date string to timestamp

from datetime import datetime, timedelta

N = 2

date_N_days_ago = datetime.now() - timedelta(weeks=N)

print(datetime.now())
print(date_N_days_ago)

2021-02-28 15:31:26.419220
2021-02-14 15:31:26.419172
